In [1]:
import os
import sys

import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

import keras
from keras.models import load_model
from keras.models import model_from_json


import cv2



Using TensorFlow backend.


In [2]:
# Importing the saved model from the IPython notebook
model = load_model('model.h5')


# Importing the Face Classifier XML file containing all features of the face
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# To open a video via link to be inserted in the () of VideoCapture()
# To open the web cam connected to your laptop/PC, write '0' (without quotes) in the () of VideoCapture()
src_cap = cv2.VideoCapture(0)


labels_dict = {
        0 : 'MASK ON',
        1 : 'NO MASK!'
    }

colorMap = {
        0 : (0,255,0),
        1 : (0,0,255)
    }


while(True):

    _ , img = src_cap.read()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # detect MultiScale / faces
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)  

    # Draw rectangles around each face
    for (x, y, w, h) in faces:
    
        #Save just the rectangle faces in SubRecFaces
        face_img = gray[y:y+w, x:x+w]

        resized = cv2.resize(face_img, (100,100))
        normalized = resized/255.0
        reshaped = np.reshape(normalized, (1,100,100,1))
        result = model.predict(reshaped)
        #print(result)

        label = np.argmax(result, axis=1)[0]
      
        cv2.rectangle(img, (x,y), (x+w,y+h), colorMap[label],2)
        cv2.rectangle(img, (x,y-40), (x+w,y), colorMap[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
        
    # Show the image
    cv2.imshow('LIVE DETECTION', img)
    key = cv2.waitKey(1)  
    
    # if Esc key (27) is press then break out of the loop
    if(key==27):
        break
        
# Stop video
src_cap.release()

# Close all started windows
cv2.destroyAllWindows()


ValueError: in user code:

    C:\Users\akhil\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\akhil\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\akhil\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\akhil\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\akhil\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x, tra